# Config

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

globals().clear()

In [2]:
path = '.../...'

In [3]:
from Module import *

# SHAP f interpretion

In [3]:
best_model = pickle.load(open(".../best_model.pkl", 'rb'))
df = pd.read_csv(".../trainingdata.csv")

In [83]:
X_ad = df.drop(['Capacity'],axis=1)
# struct 
X_non_cat1 = X_ad.drop(['HPC_index', 'name', 'metal_smiles',  'organ_smiles', 'topo', 'GLD', 'GSA', 'VF', 'Tem', 'pH', 'Dosage', 'Concentration'],axis=1)
# env
X_non_cat2 = X_ad.drop(['HPC_index', 'name', 'metal_smiles', 'organ_smiles', 'topo',  'GLD', 'PLD', 'LCD', 'ρ', 'VSA', 'GSA', 'VF', 'AV'],axis=1)
# framework1
Metal1 = X_ad.loc[:,'metal_smiles']
Frame = X_ad.loc[:,'topo']

X_cat1 = pd.concat([Metal1],axis=1)
X_cat3 = pd.concat([Frame],axis=1)
# framework2
Mfs1 = pd.read_csv('.../MF/...').loc[:, '0':] 
X_mf1 = Mfs1
X_non_cat3 = X_mf1

In [84]:
enc = OneHotEncoder()
enc.fit(X_cat1)
X_encoded1 = enc.transform(X_cat1).toarray()
feature_names1 = X_cat1.columns
new_feature_names1 = enc.get_feature_names_out(feature_names1)
X_encoded1 = pd.DataFrame(X_encoded1, columns= new_feature_names1)
X_encoded1.columns = X_encoded1.columns.str.replace('metal_smiles_', '')

In [85]:
enc = OneHotEncoder()
enc.fit(X_cat3)
X_encoded3 = enc.transform(X_cat3).toarray()
feature_names3 = X_cat3.columns
new_feature_names3 = enc.get_feature_names_out(feature_names3)
X_encoded3 = pd.DataFrame(X_encoded3, columns= new_feature_names3)
X_encoded3.columns = X_encoded3.columns.str.replace('topo_', '')

In [86]:
X = pd.concat([X_non_cat1, X_non_cat2, X_non_cat3, X_encoded1, X_encoded3],axis=1)
Y = df.loc[:,'Capacity']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state = 0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [87]:
explainer = shap.TreeExplainer(best_model)
X_sc = sc.transform(X)
X_sc = pd.DataFrame(X_sc, columns=X.columns)

In [88]:
column_mapping = {
    '[Al]': 'S1', '[Al][O](Al])[Al]': 'S2', '[Ce]': 'S3', '[Co]': 'S4', 
    '[Co][OH]1[Co][OH]2[Co]1([OH2])([OH2])[OH2][Co]2': 'S5', 
    '[Cr][O]([Cr])[Cr]': 'S6', '[Cu][O]1[Cu][O]([Cu]1)[Cu]': 'S7', '[Eu]': 'S8', 
    '[Fe]': 'S9', '[Fe][O]([Fe])[Fe]': 'S10', '[In]': 'S11', '[La]': 'S12', 
    '[Ni]': 'S13', '[O][Ce]123([O])[O]4[Ce]56([O]3[Ce]37([O]2[Ce]28([O]1[Ce]14([O]6[Ce]([O]53)([O]21)([O]78)([O])[O])([O])[O])([O])[O])([O])[O])([O])[O]': 'S14', 
    '[O][Fe][O]([Fe][O])[Fe][O]': 'S15', '[O][Zr]123[O]4[Zr]56[O]3[Zr]37([O]2[Zr]28[O]1[Zr]14([O]6[Zr]([O]53)([O]21)([O]78)[O])[O])[O]': 'S16', 
    '[O]12[Ce]34[O]5[Ce]62[O]2[Ce]71[O]4[Ce]14[O]3[Ce]35[O]6[Ce]2([O]71)[O]43': 'S17', 
    '[O]12[Zr]34[O]5[Zr]62[O]2[Zr]71[O]4[Zr]14[O]3[Zr]35[O]6[Zr]2([O]71)[O]43': 'S18', 
    '[O]12[Zr]34[OH]5[Zr]62[OH]2[Zr]71[OH]4[Zr]14[O]3[Zr]35[O]6[Zr]2([O]71)[OH]43': 'S19', 
    '[OH2][Co]([OH2])([OH2])[OH2]': 'S20', '[OH2][Yb]': 'S21', '[Pb][OH][Pb]': 'S22', 
    '[Sn][O]([Sn])[Sn]': 'S23', '[Y]': 'S24', '[Zn]': 'S25', '[Zn][O]([Zn])([Zn])[Zn]': 'S26', 
    '[Zn][OH2][Zn]': 'S27', '[Zr]': 'S28', 'O1[Ti]23O[Ti]3(O2)O[Ti]23O[Ti]3(O2)O[Ti]23[Ti](O[Ti]45[Ti]1(O4)O5)(O2)O3': 'S29'
}
X_sc.rename(columns=column_mapping, inplace=True)

In [89]:
shap_values = explainer(X_sc)

In [90]:
shap_values_array = shap_values.values
shap_values_df = pd.DataFrame(shap_values_array, columns=X_sc.columns) #total
#shap_values_df.to_csv("1-shap.csv", index=False)
mean_abs_shap_values = np.abs(shap_values.values).mean(axis=0)
mean_abs_shap_df = pd.DataFrame(mean_abs_shap_values, index=X.columns, columns=['mean_abs_shap_value']) #absolute average 
mean_abs_shap_df .to_csv("....csv", index=True)

In [ ]:
colors = ['#b2e1b9', '#77cac5', '#42a6cb', '#1373b2', '#084384'] 
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

plt.figure(figsize=(8,8))
shap.summary_plot(shap_values, X, feature_names = X.columns, class_inds='original', cmap=cmap )
plt.show()

In [ ]:
features_to_show = X_encoded3.columns#X_encoded3, X_non_cat3


feature_indices = [X.columns.get_loc(col) for col in features_to_show if col in X.columns]
shap_values_filtered = shap_values[:, feature_indices]

plt.figure(figsize=(8, 8))
shap.summary_plot(shap_values_filtered, X[features_to_show], feature_names=features_to_show, cmap=cmap)

#plt.savefig('shap2.pdf', format='pdf', dpi=300)
plt.close()

# pdp for numerical f interpretion

In [75]:
best_model = pickle.load(open(".../best_model.pkl", 'rb'))
df = pd.read_csv(".../trainingdata.csv")

In [96]:
MOF_construction = df.loc[:,'name':'topo']
Structure_info = df.loc[:,'GLD':'AV']
Absorb_env = df.loc[:,'Tem':'Concentration']
Capacity = df.loc[:,'Capacity']
Mfs1 = pd.read_csv(path2 + 'Morgan.csv').loc[:, '0':] 

Metal1 = pd.get_dummies(MOF_construction.loc[:,'metal_smiles'])
Topo = pd.get_dummies(MOF_construction.loc[:,'topo'])
Data = pd.concat([MOF_construction, Structure_info, Absorb_env, Mfs1, Metal1, Topo],axis=1) 
X = Data.drop(['name', 'metal_smiles',  'organ_smiles', 'topo', 'GLD', 'GSA', 'VF'],axis=1)
Y = Capacity

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_new_scaled = np.concatenate((X_train_scaled, X_test_scaled), axis=0)

In [97]:
Top_rank = ['PLD', 'LCD', 'ρ', 'VSA', 'AV', 'Tem', 'pH', 'Dosage', 'Concentration']
Top_rank_ind = [X.columns.get_loc(c) for c in Top_rank]  

In [ ]:
for i, idx in zip(Top_rank, Top_rank_ind):
    print(f"Feature: {i}, Index: {idx}")
    results = partial_dependence(cb_run, X_train_scaled, [idx])
    x_data = results['values'][0]
    pdp = results['average'].ravel()

    x_data_orig_scale = sc.inverse_transform(np.vstack([x_data if j == idx else np.zeros(len(x_data)) for j in range(X_train_scaled.shape[1])]).T)[:, idx]

    fig, ax1 = plt.subplots(figsize=(5, 4))
    ax1.plot(x_data_orig_scale, pdp, color='#1373b2', linewidth=3)
    ax1.set_ylabel('Partial dependence', fontsize=20)#fontweight='bold',
    ax1.set_xlabel(i, fontsize=20)

    ax1.tick_params(axis='both', labelsize=15)

    ax2 = ax1.twinx()

    sns.rugplot(X_train.iloc[:, idx], color='#1373b2', ax=ax1)
    sns.distplot(X_train.iloc[:, idx], kde=False, color='#b2e1b9', ax=ax2)
    ax2.set_ylabel('Number of data', fontsize=20)
    ax2.tick_params(axis='y', labelsize=15)

    plt.style.use('default')

    for spine in ax1.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(2)

    for spine in ax2.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(2)

    plt.show()
 
    #plt.savefig(f"pdp_{i}.png", format='png', bbox_inches='tight')
    plt.close(fig)
#print(plt.style.available)

# pearson

In [ ]:
# pearson
Xper = df.drop(['HPC_index','name', 'GLD', 'Capacity'],axis=1)
corr = Xper.corr()
pval = pd.DataFrame(index=corr.index, columns=corr.columns)

for col in corr.columns:
    for row in corr.index:
        pval.loc[row, col] = pearsonr(Xper[row], Xper[col])[1]

In [ ]:
# pearson
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(10, 10))
sns.set(font_scale=1.5)

ax = sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', linewidths=0.3, cmap='GnBu', annot_kws={"size": 13})

for i in range(len(corr.columns)):
    for j in range(i):  
        p_value = pval.iloc[i, j]
        if p_value < 0.05: 
            plt.text(j + 0.5, i + 0.2, '*', ha='center', va='center', color='black', fontsize=14)

plt.tight_layout()
plt.show()

# 2d-pdp

In [154]:
# double
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
features_to_plot = ['LCD', 'VSA', ('LCD', 'VSA')] #

In [ ]:
# double
line_kw = {"color": "#1373b2"} 
contour_kw = {"cmap": "GnBu"} 

_, ax = plt.subplots(ncols=3, figsize=(15, 4), constrained_layout=True)

tic = time.time()
display = PartialDependenceDisplay.from_estimator(
    cb_run,
    X_train_scaled_df,
    features=features_to_plot,
    kind='average',
    ax=ax,
    line_kw=line_kw,
    contour_kw=contour_kw
)
print(f"done in {time.time() - tic:.3f}s")

plt.rcParams.update({'font.size': 20})

for i, feature in enumerate(features_to_plot):
    locs = ax[i].get_xticks()  
    if isinstance(feature, str):  
        feature_idx = X_train.columns.get_loc(feature)  
        temp = np.zeros((len(locs), X_train_scaled.shape[1]))  
        temp[:, feature_idx] = locs 
        original_values = sc.inverse_transform(temp)[:, feature_idx] 
        ax[i].xaxis.set_major_locator(FixedLocator(locs))  
        ax[i].xaxis.set_major_formatter(FixedFormatter([f'{val:.0f}' for val in original_values]))  #
        ax[i].set_xlabel(feature, fontsize=15)

    elif isinstance(feature, tuple):  
        # For X-axis
        feature_idx_x = X_train.columns.get_loc(feature[0]) 
        temp_x = np.zeros((len(locs), X_train_scaled.shape[1]))  
        temp_x[:, feature_idx_x] = locs
        original_values_x = sc.inverse_transform(temp_x)[:, feature_idx_x]
        ax[i].xaxis.set_major_locator(FixedLocator(locs))
        ax[i].xaxis.set_major_formatter(FixedFormatter([f'{val:.2f}' for val in original_values_x])) #
        
        # For Y-axis
        locs_y = ax[i].get_yticks() 
        feature_idx_y = X_train.columns.get_loc(feature[1])  
        temp_y = np.zeros((len(locs_y), X_train_scaled.shape[1])) 
        temp_y[:, feature_idx_y] = locs_y
        original_values_y = sc.inverse_transform(temp_y)[:, feature_idx_y]
        ax[i].yaxis.set_major_locator(FixedLocator(locs_y))
        ax[i].yaxis.set_major_formatter(FixedFormatter([f'{val:.0f}' for val in original_values_y])) #

        ax[i].set_xlabel(feature[0], fontsize=20)
        ax[i].set_ylabel(feature[1], fontsize=20)
    ax[i].tick_params(axis='both', which='major', labelsize=15)


ax[0].set_visible(False)
ax[1].set_visible(False)

plt.show() 

# pdp for non-numerical f interpretion

In [19]:
cb_run = pickle.load(open("/Users/linzuhong/学习文件/3-博/博三/文章/2-可解释机器学习MOFs筛选-2/data/Output/XGB/XGB1_model.pkl", 'rb'))
df = pd.read_csv("/Users/linzuhong/学习文件/3-博/博三/文章/2-可解释机器学习MOFs筛选-2/data/Absorb00.csv")

In [30]:
MOF_construction = df.loc[:,'name':'topo']
Structure_info = df.loc[:,'GLD':'AV']
Absorb_env = df.loc[:,'Tem':'Concentration']
Capacity = df.loc[:,'Capacity']
Mfs1 = pd.read_csv('/Users/linzuhong/学习文件/3-博/博三/文章/2-可解释机器学习MOFs筛选/data/MF/Morgan.csv').loc[:, '0':] 

Metal1 = pd.get_dummies(MOF_construction.loc[:,'metal_smiles'])
#Metal2 = pd.get_dummies(MOF_construction.loc[:,'metal_types'])
Topo = pd.get_dummies(MOF_construction.loc[:,'topo'])
Data = pd.concat([MOF_construction, Structure_info, Absorb_env, Mfs1, Metal1, Topo],axis=1) 
X = Data.drop(['name', 'metal_smiles', 'organ_smiles', 'topo', 'GLD', 'AV','GSA'],axis=1)
Y = Capacity

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
X_train1 = X_train_df.loc[:,'O1[Ti]23O[Ti]3(O2)O[Ti]23O[Ti]3(O2)O[Ti]23[Ti](O[Ti]45[Ti]1(O4)O5)(O2)O3':'[Zr]']
rename_dict = {
    'O1[Ti]23O[Ti]3(O2)O[Ti]23O[Ti]3(O2)O[Ti]23[Ti](O[Ti]45[Ti]1(O4)O5)(O2)O3': 'A', '[Al]': 'B', '[Al][O](Al])[Al]': 'C', '[Ce]': 'D', '[Co]': 'E', '[Co][OH]1[Co][OH]2[Co]1([OH2])([OH2])[OH2][Co]2': 'F',    
    '[Cr][O]([Cr])[Cr]': 'G', '[Eu]': 'H', '[Fe]': 'I', '[Fe][O]([Fe])[Fe]': 'J', '[In]': 'K',
    '[La]': 'L', '[Ni]': 'M', '[OH2][Co]([OH2])([OH2])[OH2]': 'N', '[OH2][Yb]': 'O', '[O]12[Ce]34[O]5[Ce]62[O]2[Ce]71[O]4[Ce]14[O]3[Ce]35[O]6[Ce]2([O]71)[O]43': 'P',    
    '[O]12[Zr]34[OH]5[Zr]62[OH]2[Zr]71[OH]4[Zr]14[O]3[Zr]35[O]6[Zr]2([O]71)[OH]43': 'Q', '[O]12[Zr]34[O]5[Zr]62[O]2[Zr]71[O]4[Zr]14[O]3[Zr]35[O]6[Zr]2([O]71)[O]43': 'R',    
    '[O][Ce]123([O])[O]4[Ce]56([O]3[Ce]37([O]2[Ce]28([O]1[Ce]14([O]6[Ce]([O]53)([O]21)([O]78)([O])[O])([O])[O])([O])[O])([O])[O])([O])[O]': 'S', '[O][Fe][O]([Fe][O])[Fe][O]': 'T',    
    '[O][Fe][O]([Fe][O])[Fe][O]': 'U',  '[O][Zr]123[O]4[Zr]56[O]3[Zr]37([O]2[Zr]28[O]1[Zr]14([O]6[Zr]([O]53)([O]21)([O]78)[O])[O])[O]': 'V',  '[Pb][OH][Pb]': 'W',    
    '[Sn][O]([Sn])[Sn]': 'X', '[Y]': 'Y', '[Zn]': 'Z', '[Zn][OH2][Zn]': 'a', '[Zr]': 'b'
}
X_train1 = X_train1.rename(columns=rename_dict)

In [ ]:
X_train1

In [31]:
X_train_df.columns = X_train_df.columns.str.replace('metal_smiles_', '')
#X_train_df.columns = X_train_df.columns.str.replace('metal_types_', '')
X_train_df.columns = X_train_df.columns.str.replace('topo_', '')

X_train1 = X_train_df.loc[:,'O1[Ti]23O[Ti]3(O2)O[Ti]23O[Ti]3(O2)O[Ti]23[Ti](O[Ti]45[Ti]1(O4)O5)(O2)O3':'[Zr]']
X_train2 = X_train_df.loc[:,'acs':'sql']
#X_train3 = X_train_df.loc[:,'Al':'Zr']

features_of_interest_sets = {
    'Set1': X_train1.columns.tolist(),
    'Set2': X_train2.columns.tolist()}
    #'Set3': X_train3.columns.tolist()
#}

In [ ]:
pdp_results_all = {}
for feature in X_train_df.columns:
    pdp_res = partial_dependence(cb_run, X_train_df, features=[feature])
    pdp_results_all[feature] = pdp_res


for set_name, features in features_of_interest_sets.items():
    fig, ax1 = plt.subplots(figsize=(10, 6))
    y_pos = np.arange(len(features))
    avg_pdp_values = [pdp_results_all[feature]['average'][0][1] for feature in features if feature in pdp_results_all]
    
    if avg_pdp_values:  
        ax1.plot(y_pos, avg_pdp_values, marker='o', linestyle='-', color='#1373b2')
        ax1.set_xticks(y_pos)
        ax1.set_xticklabels(features, rotation=45, fontweight='bold', fontstyle='italic')
        ax1.set_ylabel('Partial dependence')#fontweight='bold'
        ax1.set_title(f'Partial dependence for {set_name}')
        plt.tight_layout()
        plt.show()
    else:
        print(f"No PDP data available for feature set {set_name}")